# 2.2. Ejercicio

En este ejercicio vamos a crear una Red Neuronal Convolucional para clasificar la base de datos llamada **Fashion MNIST**.

En primer lugar descargaremos la base de datos y la prepararemos para el entrenamiento. Para practicar vamos a limitar el número de muestras de entrenamiento a 50, comprobaremos el resultado obtenido e intentaremos solucionarlo de distintas formas.

Por último entrenaremos usando todos los datos originales de la base de datos y crearemos un formulario para probar cómo funciona la red al clasificar una imagen externa, subida por nosotros.

Sigue los pasos indicados y completa las líneas marcadas con "**TODO**".

## Paso 1. Descargar la base de datos

En primer lugar descargamos la base de datos y mostramos algunas imágenes escogidas aleatoriamente con su etiqueta o clase correspondiente.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import                             # TODO - importa la librería de Tensorflow como "tf"
from tensorflow.keras.datasets import fashion_mnist


tf.random.set_seed(2)  # Fijamos la semilla de TF
np.random.seed(2)  # Fijamos la semilla


# Descargamos la base de datos
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

labels = ['Camiseta/Top', 'Pantalón', 'Suéter', 'Vestido', 'Abrigo', 'Sandalia', 'Camisa', 'Zapatilla', 'Bolso/a', 'Botín']


# Mostramos algunas imágenes
n = 15
index = np.random.randint(len(x_train), size=n)
plt.figure(figsize=(n*1.5, 1.5))
for i in np.arange(n):
    ax = plt.subplot(1,n,i+1)
    ax.set_title( labels[ y_train[ index[i] ] ] )
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.imshow(x_train[index[i]], cmap='gray')
plt.show()


# Mostramos la forma de los datos
print('Datos para entrenamiento:')
print(' - x_train: {}'.format(  )) # TODO: Muestra la forma de la variable x_train
print(' - y_train: {}'.format(  )) # TODO: Muestra la forma de la variable y_train
print('Datos para evaluación:')
print(' - x_test: {}'.format(  ))   # TODO: Muestra la forma de la variable x_test
print(' - y_test: {}'.format(  ))   # TODO: Muestra la forma de la variable y_test

## Paso 2. Preparar los datos para el entrenamiento

En este paso vamos a preparar los datos para el entrenamiento. Sobre las características (x_train y x_test) tendremos que realizar una redimensión para añadir el canal de gris, transformar los datos a decimal y normalizarlos entre 0 y 1. Y las etiquetas las tendremos que transformar a modo categórico.

In [ ]:
# ---------------------
def prepare_data(x):
  x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)    # Redimensionamos para añadir el canal
  x       # TODO: Transforma la variable "x" a decimal
  x       # TODO: Normaliza la variable "x" entre 0 y 1
  return x

x_train = prepare_data(x_train)
x_test  = prepare_data(x_test)


# Transformamos las etiquetas a categórico (one-hot)
NUM_LABELS = 10
y_train =    # TODO: Transforma la variable "y_train" a categórica
y_test =     # TODO: Transforma la variable "y_test" a categórica


# Para los primeros ejemplos vamos a limitar el número de imágenes de
# entrenamiento a 50. Además nos guardamos un backup con todas las imágenes.
x_train_backup = x_train.copy()
y_train_backup = y_train.copy()
x_train = x_train[:50]
y_train = y_train[:50]


# Mostramos (de nuevo) las dimensiones de los datos
print('Datos para entrenamiento:')
print(' - x_train: {}'.format(  )) # TODO: Muestra la forma de la variable x_train
print(' - y_train: {}'.format(  )) # TODO: Muestra la forma de la variable y_train
print('Datos para evaluación:')
print(' - x_test: {}'.format(  ))   # TODO: Muestra la forma de la variable x_test
print(' - y_test: {}'.format(  ))    # TODO: Muestra la forma de la variable y_test

## Paso 3. Construimos la red y la entrenamos

En este tercer paso vamos a construir una red CNN básica y entrenarla. La red estará formada solamente por una capa convolucional (con 1 filtro de tamaño 3x3) seguida por una operación de Max Pooling (de tamaño 2x2), y por último una capa Fully Connected para la salida con 10 neuronas con activación tipo SoftMax.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model1 =   # TODO: Define un modelo secuencial

# Capa convolucional con 1 filtro de tamaño 3x3 seguida de un MaxPooling de 2x2
model1.add(Conv2D(1, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
model1.add(   )  # TODO: Añade la capa de MaxPooling2D con la configuración indicada

# Capa Fully Connected
model1.add(Flatten())
model1.add(  )  # TODO: Añade una capa densa con "NUM_LABELS" neuronas de salida y función de activación SoftMax

print(  )       # TODO: Imprime el resumen con la configuración de la red


# Compilamos la red
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

# Entrenamos durante 10 épocas con un batch de 32
history = model1.fit(x_train, y_train,
                     validation_data=(x_test, y_test),
                     batch_size= ,   # TODO - Asigna un tamaño de batch de 32
                     epochs= ,       # TODO - Asigna 10 épocas
                     verbose=1)


A continuación vamos a mostrar las curvas de aprendizaje y a evaluar el modelo entrenado con los datos de test.

In [ ]:
# -----------------------------
def plot_learning_curves(hist):
  plt.plot(hist.history['loss'])
  plt.plot(hist.history['val_loss'])
  plt.title('Curvas de aprendizaje')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Conjunto de entrenamiento', 'Conjunto de validación'], loc='upper right')
  plt.show()

plot_learning_curves(history)

# Evaluamos usando el test set
score =  model1.   # TODO: Llama a la función de evaluación del modelo entrenado con los datos x_test y y_test

print('Resultado en el test set:')
print('Test loss: {:0.4f}'.format(score[0]))
print('Test accuracy: {:0.2f}%'.format(score[1] * 100))

Como se puede ver en los resultados anteriores, parece que el modelo está haciendo overfitting: la varianza, diferencia entre el error de entrenamiento y el de validación, es muy alta. Esto también se puede ver en el accuracy obtenido, 46% para el conjunto de entrenamiento y 30% para la validación.

## Paso 4. Construimos un nuevo modelo de red

Como el resultado obtenido con el modelo anterior no es muy bueno y además parece que está haciendo overfitting, vamos a crear otro modelo de red con más filtros por cada capa convolucional, y además le añadiremos un 20% de dropout.

In [ ]:
from tensorflow.keras.layers import Dropout

model2 = Sequential()

# Capa convolucional con 64 filtros de tamaño 3x3 seguida de un MaxPooling de 2x2
model2.add(Conv2D( , (3, 3), activation='relu', input_shape=x_train.shape[1:]))   # TODO: Establece el número de filtros a 32
model2.add(  )   # TODO: Añade la capa de MaxPooling2D con la configuración indicada
model2.add(  )   # TODO: Añade un Dropout de 0.2

# Capa convolucional con 32 filtros de tamaño 3x3 seguida de un MaxPooling de 2x2
model2.add(Conv2D( , (3, 3), activation='relu'))   # TODO: Establece el número de filtros a 32
model2.add(  )   # TODO: Añade la capa de MaxPooling2D con la configuración indicada
model2.add(  )   # TODO: Añade un Dropout de 0.2

# Capa Fully Connected
model2.add(  )  # TODO: Añade una capa tipo Flatten
model2.add(Dense(NUM_LABELS, activation='softmax'))

print(model2.summary())

# Compilamos y entrenamos
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

history = model2.fit(x_train, y_train, validation_data=(x_test, y_test),
                     batch_size=  ,  # TODO: Establece el tamaño de batch a 32
                     epochs=  ,      # TODO: Establece el número de épocas a 10
                     verbose=1)


plot_learning_curves(history)

# Evaluamos usando el test set
score = model2.evaluate(x_test, y_test, verbose=0)

print('Resultado en el test set:')
print('Test loss: {:0.4f}'.format(score[0]))
print('Test accuracy: {:0.2f}%'.format(score[1] * 100))


Con esta nueva topología de red hemos conseguido mejorar el resultado, además la varianza obtenida es mucho más baja (ya no está haciendo overfitting). Sin embargo, el error de entrenamiento (el bias) sigue siendo bastante alto.

Para solucionar esto podríamos aplicar aumentado de datos (este ejercicio se deja como opcional) o añadir más datos al conjunto de entrenamiento. Como inicialmente habíamos limitado los datos vamos a usar la segunda estrategia.

## Paso 5. Entrenar con todas las imágenes

En este paso vamos a restaurar todas las imágenes de entrenamiento que nos habíamos guardado al principio en las variables `x_train_backup` y `y_train_backup`, y volveremos a lanzar el entrenamiento para el segundo modelo de red que hemos creado.

In [ ]:
# Restauramos todas las imágenes de entrenamiento
x_train = x_train_backup
y_train = y_train_backup

print('Datos para entrenamiento:')
print(' - x_train: {}'.format( x_train.shape ))
print(' - y_train: {}'.format( y_train.shape ))


# Iniciamos el entrenamiento
history = model2.fit(x_train, y_train, validation_data=(x_test, y_test),
                     batch_size=  ,  # TODO: Establece el tamaño de batch a 32
                     epochs=  ,      # TODO: Establece el número de épocas a 10
                     verbose=1)

plot_learning_curves(  )   # TODO: Pasa como parámetro a la función la variable que almacena las curvas de aprendizaje


# Evaluamos usando el test set
score = model2.evaluate(x_test, y_test, verbose=0)

print('Resultado en el test set:')
print('Test loss: {:0.4f}'.format(score[0]))
print('Test accuracy: {:0.2f}%'.format(score[1] * 100))

Como se puede ver, al entrenar con muchos más datos el resultado obtenido ha mejorado hasta alcanzar el 90% de acierto.

## Paso 6. Predicción con imágenes externas

Por último vamos a probar el funcionamiento del segundo modelo entrenado para la predicción de la clase de imágenes externas, subidas por nosotros mediante un formulario.

In [ ]:
from google.colab import files
from io import BytesIO
import cv2

uploaded = files.upload()

for fn in uploaded.keys():
  img = cv2.imread(fn, cv2.IMREAD_COLOR)
  img = 255 - cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

  plt.imshow(img)
  plt.grid(False)
  plt.show()

  # Escalamos la imagen
  img = cv2.resize(img, (28, 28))

  # Normalizamos los datos
  img = prepare_data(np.array([img]))

  # Ejecutamos la red
  prediction = model2.   # TODO: Llama a la función para calcular la predicción a partir de la variable de entrada "img"

  print('Predicción: ', labels[np.argmax(prediction)])

